In [1]:
import os
import pandas as pd

In [2]:
## CHANGE THIS FOR EACH FILE (0-45)
FILE_NO =1

## Import and Explore Dataset

In [3]:
df = pd.read_feather(str('./Data/Datasets/file' + str(FILE_NO) + '.feather'))

In [4]:
df.shape

(3510276, 15)

In [5]:
df.dtypes

marketplace           object
customer_id            int64
review_id             object
product_id            object
product_parent         int64
product_title         object
product_category      object
star_rating          float64
helpful_votes        float64
total_votes          float64
vine                  object
verified_purchase     object
review_headline       object
review_body           object
review_date           object
dtype: object

In [6]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,36075342,RAB23OVFNCXZQ,B00LPRXQ4Y,339193102,"17"" 2003-2006 Ford EXPEDITION Ford F150 2004-2...",Automotive,1.0,0.0,0.0,N,Y,"As it was used,","As it was used, the method that Ford used to a...",2015-08-31
1,US,42462164,R3NORADVJO6IE6,B000C7S0TO,907684644,Spectra Premium CU1909 Complete Radiator for T...,Automotive,5.0,0.0,0.0,N,Y,Five Stars,"Put it in fine, no problems. Shipping was dece...",2015-08-31
2,US,21241933,R299F4SO98S5OO,B000CO9WE4,752246352,K&N E-4665 High Performance Replacement Indust...,Automotive,5.0,1.0,1.0,N,Y,Great fit and performance on the surface drive...,Fit wonderfully on my 36HP Pro-Drive motor.,2015-08-31
3,US,52570308,R2DA9DOT03UW6I,B000GKD5NI,105401756,Suncutters Rear Window Shade,Automotive,5.0,2.0,3.0,N,Y,Good for the price. Fits fairly good on 2010 T...,Good for the price! So far I have put it up i...,2015-08-31
4,US,38200102,R2OGCH681EQHU6,B009SDA7TE,728471129,Lug Nuts Landcruiser Tundra OEM Mag 14x1.5 Thr...,Automotive,5.0,0.0,0.0,N,Y,Five Stars,Fit perfectly on my 2012 Tundra with stock alu...,2015-08-31


In [7]:
## Check for null
df.isnull().sum().sum()

0

In [8]:
##Check for Duplicates
data_duplicates = df[df.duplicated(subset=['product_id' ,'star_rating','review_headline', 'review_body'])]
print('\033[34m' + '\033[1m' + '\033[4m' + "\nThe number of duplicate review_body in dataset:\n" + '\033[0m' + \
    str(data_duplicates.shape[0]))


The number of duplicate review_body in dataset:
4449


In [9]:
#Drop duplicates in dataset and reset index
clean_df = df

clean_df.drop_duplicates(subset = ['product_id', 'star_rating', 'review_headline', 'review_body'], keep='first', inplace=True)
clean_df = clean_df.reset_index()

##Check for Duplicates
data_duplicates = clean_df[clean_df.duplicated(subset=['product_id' ,'star_rating','review_headline', 'review_body'])]
print('\033[34m' + '\033[1m' + '\033[4m' + "\nThe number of duplicate review_body in dataset:\n" + '\033[0m' + \
    str(data_duplicates.shape[0]))



The number of duplicate review_body in dataset:
0


# Pre-Data Preperation Calculations
## Calculate Customer Activity and Helpful/Total votes ---> Then Export to file

In [10]:
"""THESE FILES WILL BE COMBINED LATER TO CALCULATE ACTIVITY AND REPUTATION OF FULL DATASET"""

customer = clean_df.groupby('customer_id')
activity = customer['review_id'].agg('count')
helpful = customer['helpful_votes'].agg('sum')
total = customer['total_votes'].agg('sum')


customer_df = pd.DataFrame({'customer_id':activity.index, 'Activity':activity.values, 'Helpful_Votes':helpful.values, 'Total_Votes':total.values})

customer_df

,customer_id,Activity,Helpful_Votes,Total_Votes
0,10004,1,0.0,1.0
1,10014,1,1.0,2.0
2,10015,1,1.0,1.0
3,10035,1,0.0,0.0
4,10059,1,0.0,0.0
...,...,...,...,...
1904341,53096518,1,0.0,1.0
1904342,53096540,1,2.0,3.0
1904343,53096549,1,0.0,0.0
1904344,53096553,1,67.0,68.0


In [11]:
customer_df.to_feather(str('./Data/Customer_Data/customer_' + str(FILE_NO) + '.feather'))

# Data Prepertation
## Prepare data columns

In [12]:
prep_df = clean_df

In [13]:
## Remove Columns that are not to be used
prep_df = prep_df.drop(["marketplace", "product_parent", ], axis = 1)

In [14]:
##Convert Dates to Date type
prep_df['review_date'] = pd.to_datetime(prep_df['review_date'])

## Filter for Products with earliest review in 2015

In [15]:
#Group products together
product_dates = prep_df.groupby('product_id')

#Create list of the earliest review date of each product 
min_date = product_dates['review_date'].agg('min')

#Convert to dataframe
min_df = min_date.to_frame(name='Min').reset_index()

#Add year to the min_df
min_df['Year'] = pd.DatetimeIndex(min_df['Min']).year

In [16]:
#Filter for only products with the earliest review in 2015 (recent products)
#Reset the index
recent_products = min_df[min_df['Year'] == 2015]
recent_products = recent_products.reset_index(drop=True)

In [17]:
recent_products

,product_id,Min,Year
0,0983270112,2015-06-04,2015
1,0984527281,2015-01-15,2015
2,1453080279,2015-01-26,2015
3,1564137503,2015-05-03,2015
4,1609675932,2015-06-26,2015
...,...,...,...
215118,B014J838AC,2015-03-06,2015
215119,B014J8CQ9Q,2015-05-02,2015
215120,B01D42TYFC,2015-08-05,2015
215121,B01M75GUJB,2015-06-12,2015


In [18]:
#Create a reduced dataframe with only the products in recent_product dataframe
reduced_df = pd.merge(recent_products, prep_df, on="product_id", how='left')
reduced_df

,product_id,Min,Year,index,customer_id,review_id,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,0983270112,2015-06-04,2015,502145,12300008,R3DCH5Y6T4ROEW,Utah Backcountry Discovery Route Motorcycle Map,Automotive,5.0,0.0,1.0,N,Y,Five Stars,xlnt sale,2015-06-04
1,0984527281,2015-01-15,2015,274874,39921872,R3SDFQSH1IULYH,Word Teasers Learn the Truth Behind Old Wives ...,Automotive,5.0,3.0,5.0,N,Y,Five Stars,my son loves this kind of stuff and uses it da...,2015-07-15
2,0984527281,2015-01-15,2015,1222130,51019815,R1B7F0FR48QGYO,Word Teasers Learn the Truth Behind Old Wives ...,Automotive,5.0,2.0,4.0,N,Y,Five Stars,Good game,2015-01-15
3,1453080279,2015-01-26,2015,1178318,48091777,R1P313WDOPIWGO,DISNEY FROZEN GRAB & GO MINI PLAY PACK WITH CR...,Automotive,2.0,0.0,1.0,N,Y,Two Stars,Disappointing!,2015-01-26
4,1564137503,2015-05-03,2015,672552,13863876,R3RXZA4ZCMKF5W,DENSO # 3403 IRIDIUM PLATINUM power pack Spark...,Automotive,5.0,0.0,0.0,N,Y,Five Stars,On time and installed on the Tundra!!,2015-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318956,B014J838AC,2015-03-06,2015,978258,46053194,R1ZNV0DP5RSG0L,SpareCover ABC Series abc-CR-V-27-Tire Cover w...,Automotive,5.0,21.0,21.0,N,Y,GREAT US made tire cover ... Be sure your cove...,GREAT US made tire cover ... Be sure your cove...,2015-03-06
318957,B014J8CQ9Q,2015-05-02,2015,679038,46053194,R2F7HDZV2XO6KJ,SpareCover abc-Rav4-28-pink ABC Series,Automotive,5.0,15.0,16.0,N,Y,GREAT US made tire cover ... Be sure your cove...,GREAT US made tire cover ... Be sure your cove...,2015-05-02
318958,B01D42TYFC,2015-08-05,2015,157903,35353346,RR9SMJM7DGR7I,DBPOWER 600A Peak 18000mAh Portable Car Jump S...,Automotive,4.0,0.0,0.0,N,N,Battery jumper plus much more,What amazes me as it has only been a short tim...,2015-08-05
318959,B01M75GUJB,2015-06-12,2015,456956,51367435,RB9JCTHQW5VQD,Meguiar's Heavy Duty Headlight Restoration Kit.1,Automotive,5.0,0.0,0.0,Y,N,"Spray on coating saves work, looks great, and ...",G2000 and G2970 kits are identical ... both ha...,2015-06-12


## Filter for 32 <= reviews <= 100

In [19]:
##Find Number of reviews per product
#Group by products
product_reviews = reduced_df.groupby('product_id')

#Count the number of review id's (reviews) per product
product_review_count = product_reviews['review_id'].agg('count')

#Convert to dataframe and reset index
count_df = pd.DataFrame({'product_id':product_review_count.index, 'Review_count':product_review_count.values})
count_df.shape

(215123, 2)

In [20]:
#Filter out entries with less than 10 reviews
filter_count = count_df[count_df['Review_count'] >= 32]
filter_count = filter_count[filter_count['Review_count'] <= 100]
filter_count = filter_count.reset_index(drop=True)
filter_count

,product_id,Review_count
0,B000C8XRHS,15
1,B000GVDKZU,14
2,B000KZ0HYY,15
3,B000N3CP7A,13
4,B000QIQU5K,11
...,...,...
1462,B011KHEHWS,10
1463,B011Q18D14,30
1464,B012YMDPHG,11
1465,B0130NLX8G,11


In [21]:
#Create a reduced dataframe with only the products in filter_count dataframe
filter_df = pd.merge(filter_count, reduced_df, on="product_id", how='left')
filter_df

,product_id,Review_count,Min,Year,index,customer_id,review_id,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,B000C8XRHS,15,2015-02-17,2015,87322,2611776,R2B2OLPECLOLMX,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Good purchase,Bulbs came as described. Installed them and th...,2015-08-17
1,B000C8XRHS,15,2015-02-17,2015,429868,9535324,R1C9J9OPOHNLUJ,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,0.0,N,Y,"half the price of the retail product, the exac...","packaged in generic boxes, but these are the r...",2015-06-17
2,B000C8XRHS,15,2015-02-17,2015,564077,7753330,R3UCWQAU9OLH58,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,0.0,N,Y,Five Stars,"good light, very bright also",2015-05-24
3,B000C8XRHS,15,2015-02-17,2015,576953,30072229,R3E8WO60I4YNDO,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Both light up fine however. Shipping took a we...,Replace a 9005/HB3 bulb with this Wagner light...,2015-05-21
4,B000C8XRHS,15,2015-02-17,2015,578183,5146457,R3U6GK93RTV34K,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,4.0,0.0,0.0,N,Y,Four Stars,Great price.,2015-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28461,B013ME9PII,11,2015-08-16,2015,81447,46464962,R1JKAJL1912RPL,Helmet Hook,Automotive,5.0,1.0,1.0,N,N,A simple solution.,Having dropped a helmet or two in my 40+ years...,2015-08-18
28462,B013ME9PII,11,2015-08-16,2015,91506,20265694,R18FBBHIP6UD7M,Helmet Hook,Automotive,5.0,1.0,1.0,N,N,Awesome product!,"This is such a well made, clever, and simple d...",2015-08-16
28463,B013ME9PII,11,2015-08-16,2015,91698,51251197,RZ00R4KPPDQ6O,Helmet Hook,Automotive,5.0,1.0,1.0,N,N,Simple idea that works great!,This is a great little invention that solves t...,2015-08-16
28464,B013ME9PII,11,2015-08-16,2015,93005,44623952,RUVNEVCVLVEG3,Helmet Hook,Automotive,5.0,1.0,1.0,N,N,Cool idea and execution.,I bought these when they first came out a coup...,2015-08-16


## Calculate Review Order

In [22]:
##Set order Dataframe
order_df = filter_df

In [23]:
## Sort values by product then by date (earliest - oldest)
order_df = order_df.sort_values(['product_id', 'review_date'], ascending=[True, True])

In [24]:
## Generate a list of the order of reviews
prev_row = order_df['product_id'][0]
order_num = 1
order_list = []

for index, row in order_df.iterrows():
    if prev_row == row['product_id']:
        order_list.append(order_num)
        order_num += 1
        prev_row = row['product_id']
    else: 
        order_num = 1
        order_list.append(order_num)
        order_num += 1
        prev_row = row['product_id']
    

In [25]:
## Insert the list into df
order_df.insert(17, 'Review_Order', order_list)

In [26]:
order_df.head()

,product_id,Review_count,Min,Year,index,customer_id,review_id,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Review_Order
14,B000C8XRHS,15,2015-02-17,2015,1075906,50884444,RJ8BBM1YQS1ZJ,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,5.0,6.0,N,Y,Genuine Philips 9011 bulbs!,Awesome deal! I saw that there was a long dela...,2015-02-17,1
13,B000C8XRHS,15,2015-02-17,2015,940252,37453533,R1V1CATWFK58NI,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,1.0,N,Y,Five Stars,"Brighter than stock 9005, retains proper focus...",2015-03-14,2
12,B000C8XRHS,15,2015-02-17,2015,881869,36855749,R1J9I5WN2W5YM4,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Truely excellent!,Well worth the money if your looking for a bri...,2015-03-25,3
11,B000C8XRHS,15,2015-02-17,2015,870581,2586124,R30FL2MVJFRLH3,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Great deal for the price,These are actually Phillips 9011!!!<br />Great...,2015-03-27,4
10,B000C8XRHS,15,2015-02-17,2015,861058,13517324,RTBNC9X1DCQJD,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,0.0,N,Y,Great bulbs.,Work great. These are the real deal.,2015-03-29,5


## Filter Vine and Un-Verified Reviews

In [27]:
verified_df = order_df[order_df['verified_purchase'] == 'Y']

In [28]:
verified_df = verified_df[verified_df['vine'] == 'N']

In [29]:
verified_df.shape

(24282, 18)

## Calculate Helpfulness per review

In [30]:
#Function to find the helpfulness index
def helpIndex(row):
    if row['total_votes'] == 0:
        val = 0
    else:
        val = (row['helpful_votes'] / row['total_votes']) 
    return val

In [31]:
## Generate helpfulness index for each review
verified_df['helpfulness_index'] = verified_df.apply(helpIndex, axis=1)
verified_df = verified_df.round(2)
verified_df.head()

,product_id,Review_count,Min,Year,index,customer_id,review_id,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Review_Order,helpfulness_index
14,B000C8XRHS,15,2015-02-17,2015,1075906,50884444,RJ8BBM1YQS1ZJ,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,5.0,6.0,N,Y,Genuine Philips 9011 bulbs!,Awesome deal! I saw that there was a long dela...,2015-02-17,1,0.83
13,B000C8XRHS,15,2015-02-17,2015,940252,37453533,R1V1CATWFK58NI,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,1.0,N,Y,Five Stars,"Brighter than stock 9005, retains proper focus...",2015-03-14,2,0.00
12,B000C8XRHS,15,2015-02-17,2015,881869,36855749,R1J9I5WN2W5YM4,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Truely excellent!,Well worth the money if your looking for a bri...,2015-03-25,3,1.00
11,B000C8XRHS,15,2015-02-17,2015,870581,2586124,R30FL2MVJFRLH3,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,1.0,1.0,N,Y,Great deal for the price,These are actually Phillips 9011!!!<br />Great...,2015-03-27,4,1.00
10,B000C8XRHS,15,2015-02-17,2015,861058,13517324,RTBNC9X1DCQJD,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,0.0,0.0,N,Y,Great bulbs.,Work great. These are the real deal.,2015-03-29,5,0.00


## Calculate Review and Summary Length

In [32]:
#Add Review Length
verified_df['Text_Length'] = verified_df['review_body'].str.split().str.len()
verified_df['Summary_Length'] = verified_df['review_headline'].str.split().str.len()

verified_df.head()

,product_id,Review_count,Min,Year,index,customer_id,review_id,product_title,product_category,star_rating,...,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Review_Order,helpfulness_index,Text_Length,Summary_Length
14,B000C8XRHS,15,2015-02-17,2015,1075906,50884444,RJ8BBM1YQS1ZJ,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,...,6.0,N,Y,Genuine Philips 9011 bulbs!,Awesome deal! I saw that there was a long dela...,2015-02-17,1,0.83,166,4
13,B000C8XRHS,15,2015-02-17,2015,940252,37453533,R1V1CATWFK58NI,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,...,1.0,N,Y,Five Stars,"Brighter than stock 9005, retains proper focus...",2015-03-14,2,0.00,10,2
12,B000C8XRHS,15,2015-02-17,2015,881869,36855749,R1J9I5WN2W5YM4,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,...,1.0,N,Y,Truely excellent!,Well worth the money if your looking for a bri...,2015-03-25,3,1.00,22,2
11,B000C8XRHS,15,2015-02-17,2015,870581,2586124,R30FL2MVJFRLH3,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,...,1.0,N,Y,Great deal for the price,These are actually Phillips 9011!!!<br />Great...,2015-03-27,4,1.00,37,5
10,B000C8XRHS,15,2015-02-17,2015,861058,13517324,RTBNC9X1DCQJD,Wagner Lighting 9011 Halogen Capsule - Box of 1,Automotive,5.0,...,0.0,N,Y,Great bulbs.,Work great. These are the real deal.,2015-03-29,5,0.00,7,2


## Data Export

In [33]:
final_df = verified_df.sample(frac=1).reset_index(drop=True)

In [34]:
final_df.to_feather(str('./Data/Filtered_Data/filter_' + str(FILE_NO) + '.feather'))

In [35]:
## EXPORT THE NOTEBOOK TO A PDF FOR FUTURE REFERENCE
os.system("jupyter-nbconvert General_Data_Extraction_Template.ipynb --to html --output output_file" + str(FILE_NO))

0